In [66]:
import numpy as np
import pandas as pd
import scipy as sp
import pyarrow.parquet as pq
import nltk



In [67]:

table = pq.read_table("/content/train.parquet")
table2 = pq.read_table("/content/test.parquet")

In [103]:
train = table.to_pandas()
train
test = table2.to_pandas()


In [69]:
train

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id
0,1267423,Muhle Manikure Песочные колпачки для педикюра ...,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",None,None,4.666667,9,13408
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",None,5.000000,6,11790
3,1264824,Серьги,None,None,0.000000,0,14076
4,1339052,Наклейки на унитаз для туалета на крышку бачок...,"Водостойкая, интересная наклейка на унитаз раз...",None,0.000000,0,12401
...,...,...,...,...,...,...,...
283447,584544,Эфирное масло аромамасло 20мл,Аромамаркетинг – это мощный инструмент по созд...,Выберите аромат:Ваниль|Персик|Холл гостиницы|Н...,4.500000,6,2674
283448,1229689,"Форма для выпечки печенья ""Орешки""","Орешки со сгущенкой, форма для приготовления.",None,5.000000,1,13554
283449,904913,Магнит символ Нового года-Тигренок/(по 3 шт в уп),None,None,5.000000,1,11617
283450,1413201,"Рифленный нож / слайсер для фигурной нарезки, ...","Такими ножами удобно резать фрукты, овощи, сыр...","Вид:19,5х6 см",0.000000,0,14030


Анализируем данные. 
Как мы видим некоторые колонки ничем не заполнены в большенстве случаев. Можем их удалить, это может улучшить процесс обучения на данных.
Если работать с обычным текстом, то необходимо провести дополнительную очистку и лемматизацию слов, но поскольку это каталог, то можно обойтись и без этого(лемматизация слов крайне важна при обработке натурального языка, комментариев например).

In [70]:
train.describe()

,id,rating,feedback_quantity,category_id
count,2.834520e+05,283452.000000,283452.00000,283452.000000
mean,9.401805e+05,1.814641,4.34656,12244.567295
std,4.191303e+05,2.309150,34.67245,2543.823093
min,9.500000e+01,0.000000,-1.00000,2598.000000
25%,6.063808e+05,0.000000,0.00000,12049.000000
50%,9.963495e+05,0.000000,0.00000,12730.000000
75%,1.298164e+06,4.871795,2.00000,13408.000000
max,1.655762e+06,5.000000,6102.00000,14559.000000


In [71]:
train.nunique()/train.isna().sum() * 100

id                                  inf
title                               inf
short_description             69.508788
name_value_characteristics    15.154960
rating                              inf
feedback_quantity                   inf
category_id                         inf
dtype: float64

In [72]:
train = train.drop({'short_description'}, axis = 1)


Разделим датасет на тренировочную, тестовую и валидационную часть

In [73]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train ['title'], train ['category_id'], test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [74]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer делает из текста вектор.
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
# линейный классификатор и классификатор методом ближайших соседей, как раз завязан на работе с векторами
from sklearn import metrics
# метрики
from sklearn.model_selection import GridSearchCV
# с поомощью этого будем искать лучшие параметры

In [75]:
%%time
#Добавим сразу два классификатора, чтобы выбрать потом лучший из них, сравнив метрики.
sgd_gds_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])
knb_gds_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
sgd_gds_clf.fit(X_train, y_train)
knb_gds_clf.fit(X_train, y_train)

CPU times: user 4min 26s, sys: 1min 58s, total: 6min 24s
Wall time: 4min 2s


In [76]:
predicted_sgd = sgd_gds_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

        2598       0.00      0.00      0.00         0
        2599       1.00      0.69      0.82        61
        2600       0.50      0.67      0.57         3
        2601       0.04      0.67      0.08         3
        2602       0.44      0.70      0.54        10
        2603       0.00      0.00      0.00         0
        2604       0.00      0.00      0.00         0
        2605       0.00      0.00      0.00         0
        2607       0.73      0.96      0.83        49
        2608       0.10      0.50      0.17         2
        2610       0.96      0.82      0.89       113
        2631       0.86      0.84      0.85        68
        2632       0.92      0.83      0.87        29
        2633       0.33      1.00      0.50         1
        2634       0.88      0.70      0.78       141
        2635       0.52      0.85      0.65        13
        2636       0.77      0.82      0.80        50
        2662       0.51    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [77]:
predicted_sgd = knb_gds_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

        2598       0.00      0.00      0.00         0
        2599       0.98      0.79      0.87        52
        2600       1.00      0.57      0.73         7
        2601       0.34      0.39      0.36        41
        2602       0.44      0.58      0.50        12
        2603       0.00      0.00      0.00         1
        2604       0.00      0.00      0.00         0
        2605       0.43      0.64      0.51        14
        2607       0.84      0.72      0.78        75
        2608       0.50      0.45      0.48        11
        2610       0.87      0.81      0.84       104
        2631       0.89      0.86      0.87        69
        2632       0.96      0.78      0.86        32
        2633       0.67      1.00      0.80         2
        2634       0.71      0.49      0.58       164
        2635       0.48      0.59      0.53        17
        2636       0.60      0.58      0.59        55
        2662       0.68    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

Лучше всего для оценки несбалансированных классов подойдет метрика precision, recall, или их гормоническое среднее (f1-score).Так как когда много различных классов accuracy будет выдавать неадекватную оценку.
Как мы видим SGD показал лучшую метрику (weighted avg - 80.00)

Подбирая различные параметры я понял, что эти являются оптимальными.С ними и предскажем категории.

In [148]:
X_test = test['title']

In [149]:
solution = sgd_gds_clf.predict(X_test)

In [150]:
solution_df = pd.DataFrame(solution, columns = {'predicted_category_id'})

In [151]:
solution_df = pd.DataFrame.join(test,solution_df)

In [152]:
solution = solution_df[{'id', 'predicted_category_id'}]

In [159]:
solution.to_parquet('/content/result.parquet')

,id,predicted_category_id
0,1070974,11574
1,450413,11878
2,126857,13299
3,1577569,13061
4,869328,12813
...,...,...
70859,967535,13143
70860,1488636,2674
70861,827510,2804
70862,529244,2599


,id,predicted_category_id
0,1070974,11574
1,450413,11878
2,126857,13299
3,1577569,13061
4,869328,12813
...,...,...
70859,967535,13143
70860,1488636,2674
70861,827510,2804
70862,529244,2599
